In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

def sift(filename):
    # img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)  # 读取文件
    img = cv2.imread(filename)  # 读取文件
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 转化为灰度图
    sift = cv2.SIFT.create()
    keyPoint, descriptor = sift.detectAndCompute(img, None)  # 特征提取得到关键点以及对应的描述符（特征向量）
    return img, keyPoint, descriptor


def surf(filename):
    img = cv2.imread(filename)  # 读取文件
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 转化为灰度图
    sift = cv2.xfeatures2d_SURF.create()
    keyPoint, descriptor = sift.detectAndCompute(img, None)  # 特征提取得到关键点以及对应的描述符（特征向量）
    return img, keyPoint, descriptor


def orb(filename):
    # img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)  # 读取文件
    img = cv2.imread(filename)  # 读取文件
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 转化为灰度图
    orb = cv2.ORB.create()
    keyPoint, descriptor = orb.detectAndCompute(img, None)  # 特征提取得到关键点以及对应的描述符（特征向量）
    return img, keyPoint, descriptor




def stitch(filename1, filename2):
    img1, kp1, des1 = sift(filename1)
    img2, kp2, des2 = sift(filename2)
    bf = cv2.BFMatcher(cv2.NORM_L2)  # sift的normType应该使用NORM_L2或者NORM_L1
    # img1, kp1, des1 = orb(filename1)
    # img2, kp2, des2 = orb(filename2)
    # bf = cv2.BFMatcher(cv2.NORM_HAMMING)  # orb的normType应该使用NORM_HAMMING
    rawMatches = bf.knnMatch(des1, des2,2)# drawMatchesKnn
    matches = []
    for m in rawMatches:
        # 当最近距离跟次近距离的比值小于ratio值时，保留此匹配对
        if len(m) == 2 and m[0].distance < m[1].distance * 0.75:
        # 存储两个点在featuresA, featuresB中的索引值
            matches.append((m[0].trainIdx, m[0].queryIdx))
        # 当筛选后的匹配对大于4时，计算视角变换矩阵
    if len(matches) > 4:
        # 获取匹配对的点坐标
        ptsA = np.float32([kp1[i].pt for (_, i) in matches])
        ptsB = np.float32([kp2[i].pt for (i, _) in matches])
        # 计算视角变换矩阵
        H, status = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, 4.0)
        # 返回结果
        result = cv2.warpPerspective(img1, H, (img1.shape[1] + img2.shape[1], img1.shape[0]))
        # cv2.imshow('result', result)
        # cv2.waitKey(0)
        # 将图片B传入result图片最左端
        result[0:img2.shape[0], 0:img2.shape[1]] = img2
        
     
        # 未处理接缝
        cv2.imshow('result', result)
        cv2.waitKey(0)
        
        return None
    # 如果匹配对小于4时，返回None
    return None

imageDir='images'
for item in os.listdir(imageDir):
    item_path = os.path.join(imageDir, item)
    if os.path.isdir(item_path):
        stitch(os.path.join(item_path,'1.jpg'), os.path.join(item_path,'2.jpg'))



In [21]:
import os
for item in os.listdir('images'):
    print(item)

p1
p2
